### 지형 데이터 2. 집계구 별로 매칭시키기

In [3]:
import pandas_gbq as pd_gbq
import pandas as pd

In [4]:
# 1 process 를 통해 산출된 데이터
df = pd.read_csv('/Users/yeonjin/Documents/GitHub/Market-Entry-Analysis/topography_data/tf_seoul/tfseoul_1.csv')

/Users/yeonjin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,...,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자,standard_date
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2019,1,8,...,아파트,2,평지,4,사다리형,6,소로한면,3961000,2019-08-08,2019-01-01
1,1111010100100010001,1111010100,서울특별시 종로구 청운동,1.0,일반,1-1,2,2019,1,14,...,도로등,2,평지,7,부정형,0,지정되지않음,1296000,2019-08-08,2019-01-01
2,1111010100100010002,1111010100,서울특별시 종로구 청운동,1.0,일반,1-2,3,2019,1,8,...,주거나지,3,완경사,7,부정형,12,맹지,3267000,2019-08-08,2019-01-01
3,1111010100100010003,1111010100,서울특별시 종로구 청운동,1.0,일반,1-3,4,2019,1,8,...,연립,2,평지,7,부정형,6,소로한면,4255000,2019-08-08,2019-01-01
4,1111010100100010004,1111010100,서울특별시 종로구 청운동,1.0,일반,1-4,5,2019,1,14,...,도로등,2,평지,7,부정형,0,지정되지않음,1524000,2019-08-08,2019-01-01


In [6]:
#df['%서대문구%' in df['법정동명']]
df[df['법정동명'].str.contains("서대문구")].head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,...,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자,standard_date
490364,1141010100100010001,1141010100,서울특별시 서대문구 충정로2가,1.0,일반,1-1,1,2019,1,14,...,도로등,0,지정되지않음,0,지정되지않음,0,지정되지않음,1593000,2019-08-08,2019-01-01
490365,1141010100100020002,1141010100,서울특별시 서대문구 충정로2가,1.0,일반,2-2,2,2019,1,8,...,업무용,2,평지,2,가로장방,3,광대세각,19150000,2019-08-08,2019-01-01
490366,1141010100100050002,1141010100,서울특별시 서대문구 충정로2가,1.0,일반,5-2,3,2019,1,14,...,도로등,0,지정되지않음,0,지정되지않음,0,지정되지않음,2603000,2019-08-08,2019-01-01
490367,1141010100100050004,1141010100,서울특별시 서대문구 충정로2가,1.0,일반,5-4,4,2019,1,8,...,업무용,2,평지,7,부정형,3,광대세각,17360000,2019-08-08,2019-01-01
490368,1141010100100050006,1141010100,서울특별시 서대문구 충정로2가,1.0,일반,5-6,5,2019,1,8,...,상업용,2,평지,4,사다리형,8,세로한면(가),8490000,2019-08-08,2019-01-01


## 행정동코드 vs 법정동코드 - 토지 관련 데이터는 주로 법정동코드를 사용한다고 함

In [7]:
# 행정동코드 
url = "http://sgis.kostat.go.kr/OpenAPI2/upload/unitarea_code.xls"
hjd = pd.read_excel(url, skiprows=1)
hjd.head()

,시도코드,시도명칭,시군구코드,시군구명칭,읍면동코드,읍면동명칭
0,11,서울특별시,11010,종로구,1101053,사직동
1,11,서울특별시,11010,종로구,1101054,삼청동
2,11,서울특별시,11010,종로구,1101055,부암동
3,11,서울특별시,11010,종로구,1101056,평창동
4,11,서울특별시,11010,종로구,1101057,무악동


In [9]:
# 법정동코드
bjd = pd.read_csv('/Users/yeonjin/Documents/GitHub/Market-Entry-Analysis/topography_data/raw/법정동코드.txt', sep = '\t')

In [10]:
ss = []
for i in bjd['법정동코드'].astype(str):
    ss.append(i[:5])

In [11]:
bjd['법정동코드_구'] = ss

In [12]:
k = []
for i in bjd['법정동명']:
    k.append(str(i)[-1])

In [13]:
bjd['구'] = k

In [14]:
pd.DataFrame(k).drop_duplicates()

,0
0,시
1,구
2,동
20,로
21,가
1431,리
2297,)
2833,군
2834,읍
2875,면


In [15]:
gu_listup = bjd[bjd['구'] == '구'][['법정동명', '법정동코드_구']]

In [16]:
seoul_gu_listup = gu_listup[ bjd['법정동명'].str.contains('서울특별시') ]

/Users/yeonjin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [17]:
seoul_gu_listup.columns = ['법정동구', '법정동코드_구']

## 서울시 지형 데이터에서 법정동코드를 별도로 제작하기

In [18]:
df['법정동코드']  = df['법정동코드'].astype(str)
#df.dtypes

In [19]:
df['법정동코드_구'] = df['법정동코드'].str.slice(start=0, stop=5)

In [20]:
df.head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,...,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자,standard_date,법정동코드_구
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2019,1,8,...,2,평지,4,사다리형,6,소로한면,3961000,2019-08-08,2019-01-01,11110
1,1111010100100010001,1111010100,서울특별시 종로구 청운동,1.0,일반,1-1,2,2019,1,14,...,2,평지,7,부정형,0,지정되지않음,1296000,2019-08-08,2019-01-01,11110
2,1111010100100010002,1111010100,서울특별시 종로구 청운동,1.0,일반,1-2,3,2019,1,8,...,3,완경사,7,부정형,12,맹지,3267000,2019-08-08,2019-01-01,11110
3,1111010100100010003,1111010100,서울특별시 종로구 청운동,1.0,일반,1-3,4,2019,1,8,...,2,평지,7,부정형,6,소로한면,4255000,2019-08-08,2019-01-01,11110
4,1111010100100010004,1111010100,서울특별시 종로구 청운동,1.0,일반,1-4,5,2019,1,14,...,2,평지,7,부정형,0,지정되지않음,1524000,2019-08-08,2019-01-01,11110


In [21]:
ndf = pd.merge(df, seoul_gu_listup, how = 'left')

In [22]:
ndf.head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,...,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자,standard_date,법정동코드_구,법정동구
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2019,1,8,...,평지,4,사다리형,6,소로한면,3961000,2019-08-08,2019-01-01,11110,서울특별시 종로구
1,1111010100100010001,1111010100,서울특별시 종로구 청운동,1.0,일반,1-1,2,2019,1,14,...,평지,7,부정형,0,지정되지않음,1296000,2019-08-08,2019-01-01,11110,서울특별시 종로구
2,1111010100100010002,1111010100,서울특별시 종로구 청운동,1.0,일반,1-2,3,2019,1,8,...,완경사,7,부정형,12,맹지,3267000,2019-08-08,2019-01-01,11110,서울특별시 종로구
3,1111010100100010003,1111010100,서울특별시 종로구 청운동,1.0,일반,1-3,4,2019,1,8,...,평지,7,부정형,6,소로한면,4255000,2019-08-08,2019-01-01,11110,서울특별시 종로구
4,1111010100100010004,1111010100,서울특별시 종로구 청운동,1.0,일반,1-4,5,2019,1,14,...,평지,7,부정형,0,지정되지않음,1524000,2019-08-08,2019-01-01,11110,서울특별시 종로구


In [69]:
#ndf.to_csv('/Users/yeonjin/Documents/GitHub/Market-Entry-Analysis/topography_data/tf_seoul/tf_seoul_200617.csv', index = False)

In [70]:
#pd.read_csv('/Users/yeonjin/Documents/GitHub/Market-Entry-Analysis/topography_data/tf_seoul/tf_seoul_200617.csv').head()

/Users/yeonjin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,...,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자,standard_date,법정동코드_구,법정동구
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2019,1,8,...,평지,4,사다리형,6,소로한면,3961000,2019-08-08,2019-01-01,11110,서울특별시 종로구
1,1111010100100010001,1111010100,서울특별시 종로구 청운동,1.0,일반,1-1,2,2019,1,14,...,평지,7,부정형,0,지정되지않음,1296000,2019-08-08,2019-01-01,11110,서울특별시 종로구
2,1111010100100010002,1111010100,서울특별시 종로구 청운동,1.0,일반,1-2,3,2019,1,8,...,완경사,7,부정형,12,맹지,3267000,2019-08-08,2019-01-01,11110,서울특별시 종로구
3,1111010100100010003,1111010100,서울특별시 종로구 청운동,1.0,일반,1-3,4,2019,1,8,...,평지,7,부정형,6,소로한면,4255000,2019-08-08,2019-01-01,11110,서울특별시 종로구
4,1111010100100010004,1111010100,서울특별시 종로구 청운동,1.0,일반,1-4,5,2019,1,14,...,평지,7,부정형,0,지정되지않음,1524000,2019-08-08,2019-01-01,11110,서울특별시 종로구
